In [ ]:
import networkx as nx
import numpy as np
import gsgmorph.matrix_form
import timeit
import matplotlib.pyplot as plt
import csv

In [ ]:
rng = np.random.default_rng()

with open('edge_benchmark.csv', 'w', newline='') as csvfile:
    fieldnames = ['edge_prob', 'gsgmorph', 'hua', 'qubo_equal']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    # fixed number of nodes:
    nodes = 50
    # prob. of edge generation goes in increments of 5%
    for prob_edges in range(0,105,5):
        g1 = nx.erdos_renyi_graph(nodes, prob_edges/100, seed=0)

        # number of times to shuffle the nodes around
        gsgmorph_times = []
        hua_times = []

        print("\rEdge Prob: {0}\r".format(prob_edges), end='')

        # Keep the edges the same but shuffle the nodes around
        perm = rng.permutation(g1.nodes())
        mapping = {i: v for i, v in zip(g1.nodes(), perm)}
        g2 = nx.relabel_nodes(g1, mapping, copy=True)
        def gsgm_graph_isomorphism():
            gsgmorph.matrix_form.integer_graph_isomorphism(g1,g2)
        def hua_graph_isomorphism():
            gsgmorph.matrix_form.hua_graph_isomorphism(g1,g2)
        # `timeit` arguments: https://stackoverflow.com/questions/56763416/what-is-diffrence-between-number-and-repeat-in-python-timeit
        # number: run the program back-to-back 
        # repeat: repeat * number (see line above)
        gsgmorph_time = min(timeit.repeat(gsgm_graph_isomorphism, repeat=2, number=4))
        hua_time = min(timeit.repeat(hua_graph_isomorphism, repeat=2, number=4))

        gsgmorph_matrix = gsgmorph.matrix_form.integer_graph_isomorphism(g1, g2) 
        hua_matrix = gsgmorph.matrix_form.hua_graph_isomorphism(g1, g2)
        matrix_equal = ""
        if(gsgmorph_matrix == hua_matrix):
            matrix_equal = "TRUE"
        else:
            matrix_equal = "FALSE"
            
        writer.writerow({'edge_prob':prob_edges, 
                         'gsgmorph': gsgmorph_time, 
                         'hua':hua_time,
                         'qubo_equal':matrix_equal})